In [1]:
!pip install opencv-python
!pip install pandas

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import cv2 as cv
import tqdm

def compress_images_in_folder(folder_path):
    # フォルダーが存在する場合の例外処理
    new_folder_path = folder_path + "_new"
    try:
        os.makedirs(new_folder_path)
    except FileExistsError:
        pass

    img_list = os.listdir(folder_path)
    for i in tqdm.tqdm(img_list):
        img_path = os.path.join(folder_path, i)
        if os.path.isdir(img_path):
            # This is a subdirectory, recurse into it
            compress_images_in_folder(img_path)
            continue
        elif not os.path.isfile(img_path):
            print(f"Skipped {img_path}, not a file")
            continue

        img = cv.imread(img_path)
        if img is None:
            print(f"Skipped {img_path}, cannot read the image")
            continue

        scale_factor = 1.0
        while scale_factor > 0:
            # Start with high quality
            quality = 90
            new_img_path = os.path.join(new_folder_path, i)
            while quality > 0:
                # Resize the image
                new_img = cv.resize(img, None, fx=scale_factor, fy=scale_factor)
                # Save with reduced quality
                cv.imwrite(new_img_path, new_img, [int(cv.IMWRITE_JPEG_QUALITY), quality])
                # Check file size
                file_size_kb = os.path.getsize(new_img_path) / 1024
                if file_size_kb <= 100:
                    break
                quality -= 10  # Reduce quality

            if file_size_kb <= 100:
                break

            scale_factor -= 0.1  # Reduce scale factor

        if scale_factor <= 0:
            print(f"Could not compress {img_path} to less than 100KB")

# Start compression in the root folder
compress_images_in_folder("/<your-folder-for-compression>")


100%|██████████| 1/1 [00:00<00:00,  1.14it/s]
